In [1]:
import numpy as np
import torch

from transformers import AutoModelForMultipleChoice, AutoTokenizer
from datasets import load_dataset

from src.utils_multiple_choice import convert_examples_to_features, InputExample

In [2]:
model = AutoModelForMultipleChoice.from_pretrained("../assets/models/bb_race_m/")
tokenizer = AutoTokenizer.from_pretrained("../assets/models/bb_race_m")

dataset = load_dataset("race", "middle")
test = dataset['test']

label_list = ["0", "1", "2", "3"]
label_map = {
    0: "A",
    1: "B",
    2: "C",
    3: "D"
}
max_seq_length = 128

Reusing dataset race (/home/marcos/.cache/huggingface/datasets/race/middle/0.1.0/a7d1fac780e70c0e75bca35e9f2f8cfc1411edd18ffd6858ddce56f70dfb1e7c)


In [3]:
def predict(article_, question_, options_, real_label=None):
    examples = [InputExample(
        example_id="pred",
        question=question_,
        contexts=[article_, article_, article_, article_],  # this is not efficient but convenient
        endings=[options_[0], options_[1], options_[2], options_[3]],
        label=str(ord(real_label) - ord("A")) if real_label else "0"
    )]
    
    feature = convert_examples_to_features(
        examples,
        label_list,
        max_seq_length,
        tokenizer
    )[0]
    
    features = {
        'input_ids': torch.tensor([feature.input_ids]),
        'attention_mask': torch.tensor([feature.attention_mask]),
        'token_type_ids': torch.tensor([feature.token_type_ids]),
    } 
    
    result = model.forward(features['input_ids'], features['attention_mask'], features['token_type_ids'])[0][0]
    
#     print(f"Scores: {[float(abs(x)) for x in result]}")
    
    return np.array([float(abs(x)) for x in result]).argmax()

### Example 1

In [4]:
ex = test[0]

article = ex['article']
question = ex['question']
options = ex['options']
real_label = ex['answer']

result = predict(article, question, options, real_label)
print(f"Article: {article}")
print(f"Question: {question}")
print(f"Options: {options}")
print(f"Prediction: {label_map[result]}")
print(f"Real answer: {real_label}")

convert examples to features: 1it [00:00, 67.80it/s]


Article: Take a class at Dulangkou School, and you'll see lots of things different from other schools, You can see the desks are not in rows and students sit in groups. They put their desks together so they're facing each other. How can they see the blackboard? There are three blackboards on the three walls of the classroom!
The school calls the new way of learning "Tuantuanzuo", meaning sitting in groups. Wei Liying, a Junior 3 teacher, said it was to give students more chances to communicate.
Each group has five or six students, according to Wei, and they play different roles .There is a team leader who takes care of the whole group. There is a "study leader"who makes sure that everyone finishes their homework. And there is a discipline leader who makes sure that nobody chats in class.
Wang Lin is a team leader. The 15-year-old said that having to deal with so many things was tiring.
"I just looked after my own business before,"said Wang. "But now I have to think about my five group 

##### Mod a

In [5]:
ex = test[0]

article = ex['article']
question = "What is a discipline leader supposed to?"
real_label = ex['answer']

result = predict(article, question, options, real_label)
# print(f"Article: {article}")
print(f"Question: {question}")
print(f"Options: {options}")
print(f"Prediction: {label_map[result]}")
print(f"Real answer: {real_label}")

convert examples to features: 1it [00:00, 66.03it/s]


Question: What is a discipline leader supposed to?
Options: ['take care of the whole group', 'make sure that everybody finishes homework', 'make sure that nobody chats in class', 'collect all the homework and hand it in to teachers']
Prediction: A
Real answer: C


##### Mod b

In [6]:
ex = test[0]

article = ex['article']
question = "What is a discipline leader?"
options = [
    "A person supposed to take care of the whole group",
    "A person supposed to make sure that everybody finished homework",
    "A person supposed to make sure that nobody chats in class",
    "A person supposed to collect all the homework and hand it in to teachers"
]
real_label = ex['answer']

result = predict(article, question, options, real_label)
# print(f"Article: {article}")
print(f"Question: {question}")
print(f"Options: {options}")
print(f"Prediction: {label_map[result]}")
print(f"Real answer: {real_label}")

convert examples to features: 1it [00:00, 83.15it/s]


Question: What is a discipline leader?
Options: ['A person supposed to take care of the whole group', 'A person supposed to make sure that everybody finished homework', 'A person supposed to make sure that nobody chats in class', 'A person supposed to collect all the homework and hand it in to teachers']
Prediction: B
Real answer: C


##### Mod c

In [7]:
ex = test[0]

article = ex['article']
question = "What is an orderliness leader?"
options = [
    "A person supposed to take care of the whole group",
    "A person supposed to make sure that everybody finished homework",
    "A person supposed to make sure that nobody chats in class",
    "A person supposed to collect all the homework and hand it in to teachers"
]
real_label = ex['answer']

result = predict(article, question, options, real_label)
# print(f"Article: {article}")
print(f"Question: {question}")
print(f"Options: {options}")
print(f"Prediction: {label_map[result]}")
print(f"Real answer: {real_label}")

convert examples to features: 1it [00:00, 83.51it/s]


Question: What is an orderliness leader?
Options: ['A person supposed to take care of the whole group', 'A person supposed to make sure that everybody finished homework', 'A person supposed to make sure that nobody chats in class', 'A person supposed to collect all the homework and hand it in to teachers']
Prediction: C
Real answer: C


### Example 2

In [8]:
ex = test[86]

article = ex['article']
question = ex['question']
options = ex['options']
real_label = ex['answer']

result = predict(article, question, options, real_label)
print(f"Article: {article}")
print(f"Question: {question}")
print(f"Options: {options}")
print(f"Prediction: {label_map[result]}")
print(f"Real answer: {real_label}")

convert examples to features: 1it [00:00, 97.79it/s]


Article: A traveler came out of the airport. There were a lot of taxis. He asked every taxi driver about his name. Then he took the third one. It cost 5 dollars from the airport to the hotel. "How much does it cost for the whole day?" The man asked. "100 dollars," said the taxi driver. This was very dear  , but the man said it was OK.
The taxi driver took the man everywhere. He showed him all the parks and museums in the city. In the evening they went back to the hotel. The traveler gave the taxi driver 100 dollars and said, "What about tomorrow?" The taxi driver looked at the man and said, "Tomorrow is another 100 dollars." And the man said, "That's OK! See you tomorrow." The taxi driver was very pleased.
The next day the taxi driver took the traveler everywhere again. They visited all the parks and museums again. And in the evening they went back to the hotel. The man gave the taxi driver 100 dollars again and said, "I'm going home tomorrow." The driver was sorry because he liked the

##### Mod a

In [9]:
ex = test[86]

article = ex['article']
question = "The traveler came from _ ."
options = ex['options']
real_label = ex['answer']

result = predict(article, question, options, real_label)
print(f"Question: {question}")
print(f"Options: {options}")
print(f"Prediction: {label_map[result]}")
print(f"Real answer: {real_label}")

convert examples to features: 1it [00:00, 97.99it/s]


Question: The traveler came from _ .
Options: ['England', 'America', 'Canada', 'France']
Prediction: C
Real answer: B


##### Mod b

In [10]:
ex = test[86]

article = ex['article']
question = "The traveler _ ."
options = [
    "came from England",
    "came from America",
    "came from Canada",
    "came from France"
]
real_label = ex['answer']

result = predict(article, question, options, real_label)
print(f"Question: {question}")
print(f"Options: {options}")
print(f"Prediction: {label_map[result]}")
print(f"Real answer: {real_label}")

convert examples to features: 1it [00:00, 72.88it/s]


Question: The traveler _ .
Options: ['came from England', 'came from America', 'came from Canada', 'came from France']
Prediction: C
Real answer: B


##### Mod c

In [11]:
ex = test[86]

article = ex['article']
question = "The visitor _ ."
options = [
    "came from England",
    "came from America",
    "came from Canada",
    "came from France"
]
real_label = ex['answer']

result = predict(article, question, options, real_label)
print(f"Question: {question}")
print(f"Options: {options}")
print(f"Prediction: {label_map[result]}")
print(f"Real answer: {real_label}")

convert examples to features: 1it [00:00, 79.72it/s]


Question: The visitor _ .
Options: ['came from England', 'came from America', 'came from Canada', 'came from France']
Prediction: C
Real answer: B
